In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
WATSONX_API_KEY = os.environ["WATSONX_API_KEY"]
WATSONX_SPACE_ID = os.environ["WATSONX_SPACE_ID"]
WATSONX_DEPLOYMENT_ID = os.environ["WATSONX_DEPLOYMENT_ID"]

## CALLING METHOD 1 directly

In [10]:
import requests

# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account (https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html?context=wx)
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey":
 WATSONX_API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}


payload_scoring = {
    "messages": [
        {
            "role": "user",
            "content": "การลามีกี่ประเภท"
        }
    ]
}

response_scoring = requests.post(f'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/{WATSONX_DEPLOYMENT_ID}/ai_service?version=2021-05-01', json=payload_scoring,
 headers={'Authorization': 'Bearer ' + mltoken})

print("Scoring response")
try:
    print(response_scoring.json())
except ValueError:
    print(response_scoring.text)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Scoring response
{'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': 'จากข้อมูลในนโยบายการลาประจำปีของบริษัท มีการระบุประเภทของการลาที่ได้รับการอนุมัติดังนี้:\n\n1. การลาชั่วคราว\n2. การลาที่ได้รับ (การลาพักร้อนที่ได้รับ)\n3. การลาคลอด\n4. การลาพักผ่อนสำหรับบิดา\n5. การลาโดยไม่ได้รับค่าจ้าง\n6. วันหยุดชดเชย\n\nดังนั้น ประเภทของการลาที่ระบุไว้ในนโยบายมีทั้งหมด 6 ประเภท', 'role': 'assistant'}}], 'created': 1747717323, 'created_at': '2025-05-20T05:02:06.419Z', 'id': 'chatcmpl-54d4ad647a377d5f1c65b78bc42fd1fc', 'model': 'mistralai/mistral-medium-2505', 'model_id': 'mistralai/mistral-medium-2505', 'model_version': '1.0.0', 'object': 'chat.completion', 'system': {'warnings': [{'id': 'disclaimer_warning', 'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/c

In [11]:
print(response_scoring.json()["choices"][0]["message"]["content"])

จากข้อมูลในนโยบายการลาประจำปีของบริษัท มีการระบุประเภทของการลาที่ได้รับการอนุมัติดังนี้:

1. การลาชั่วคราว
2. การลาที่ได้รับ (การลาพักร้อนที่ได้รับ)
3. การลาคลอด
4. การลาพักผ่อนสำหรับบิดา
5. การลาโดยไม่ได้รับค่าจ้าง
6. วันหยุดชดเชย

ดังนั้น ประเภทของการลาที่ระบุไว้ในนโยบายมีทั้งหมด 6 ประเภท


## CALLING METHOD 2 using SDK

In [13]:
import os
from ibm_watsonx_ai import APIClient, Credentials
import getpass

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=WATSONX_API_KEY
)

client = APIClient(credentials)

client.set.default_space(WATSONX_SPACE_ID)

'SUCCESS'

In [17]:
# deployment_id
# messages = []
remote_question = "ผมชื่อว่าอะไรครับ"
# messages = [{"role":"user","content":"พนักงานเข้าทำงานกี่โมง"}]
messages = [{ "role" : "user", "content": "สวัสดีครับ ผมชื่อมิว" }, { "role" : "assistant", "content": "สวัสดีครับ มิวต้องการให้ผมช่วยอะไรครับ ผมคือผู้ช่วยพนักงาน" }, { "role" : "user", "content": remote_question }]
payload = { "messages": messages }
result = client.deployments.run_ai_service(WATSONX_DEPLOYMENT_ID, payload)
if "error" in result:
    print(result["error"])
else:
    print(result)

{'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': 'จากการสนทนา ผมได้ชื่อของคุณว่า มิวครับ', 'role': 'assistant'}}], 'created': 1747717520, 'created_at': '2025-05-20T05:05:20.849Z', 'id': 'chatcmpl-c5e3d1dc0d2c86ed78f0263b59fdcc62', 'model': 'mistralai/mistral-medium-2505', 'model_id': 'mistralai/mistral-medium-2505', 'model_version': '1.0.0', 'object': 'chat.completion', 'system': {'warnings': [{'id': 'disclaimer_warning', 'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}, 'usage': {'completion_tokens': 19, 'prompt_tokens': 2932, 'total_tokens': 2951}}


In [18]:
print(result["choices"][0]["message"]["content"])

จากการสนทนา ผมได้ชื่อของคุณว่า มิวครับ
